In [1]:
import opendatasets as od

In [ ]:
od.download('https://www.kaggle.com/datasets/wyattowalsh/basketball')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/wyattowalsh/basketball


 16%|█▋        | 114M/697M [04:57<21:00, 485kB/s] 